In [1]:
from pathlib import Path
import copy

In [2]:
test_input = """498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
"""

In [3]:
SAND_ENTRYPOINT = (500, 0)

def parse_input(rock_location, floor=False):
    rock_formations = [
        [tuple(map(int, element.split(","))) for element in row.split(" -> ")]
        for row in rock_location.strip().split("\n")
    ]
    
    max_y = max(point[1] for formation in rock_formations for point in formation)
    max_x = max(point[0] for formation in rock_formations for point in formation)
    min_x = min(point[0] for formation in rock_formations for point in formation)
    
    if floor:
        max_y += 2
        min_x = min((500 - (max_y + max_y-1) // 2) - 1, max_x)
        max_x = max((500 + (max_y + max_y-1) // 2) + 1, max_x)
    
    rock_map = [["."] * (max_x - min_x + 1) for y in range(max_y + 1)]
    for formation in rock_formations:
        for n, point in enumerate(formation[1:]):
            previous_point = formation[n]
            x1, x2 = previous_point[0], point[0]
            if x1 > x2:
                x1, x2 = x2, x1
                
            y1, y2 = previous_point[1], point[1]
            if y1 > y2:
                y1, y2 = y2, y1
            
            for y in range(y1, y2 + 1):
                for x in range(x1, x2 + 1):
                    x -= min_x
                    rock_map[y][x] = "#"
    if floor:
        for x in range(max_x + 1 - min_x):
            rock_map[-1][x] = "#"
    return rock_map, min_x      

def new_sand_unit(rock_map, x_offset):
    x, y = SAND_ENTRYPOINT
    x -= x_offset
    
    sand_in_motion = True
    while sand_in_motion:
        if y == len(rock_map) - 1 or x in (0, len(rock_map[0])-1):
            break
        if rock_map[y + 1][x] == ".":
            y += 1
        elif rock_map[y + 1][x - 1] == ".":
            x -= 1
            y += 1
        elif rock_map[y + 1][x + 1] == ".":
            x += 1
            y += 1
        else:
            sand_in_motion = False
            rock_map[y][x] = "o"  
    return rock_map

def new_sand_unit_detect_freefall(rock_map, x_offset):
    initial_rock_map = copy.deepcopy(rock_map)
    new_rock_map = new_sand_unit(rock_map, x_offset)
    return new_rock_map, new_rock_map == initial_rock_map

    
rock_map, x_offset = parse_input(test_input)
assert len(rock_map) == 10
assert len(rock_map[0]) == 10

rock_map, _ = new_sand_unit_detect_freefall(rock_map, x_offset)
assert rock_map[8][500-x_offset] == "o"

rock_map, _ = new_sand_unit_detect_freefall(rock_map, x_offset)
assert rock_map[8][499-x_offset] == "o"

In [4]:
# Part 1 - test

rock_map, x_offset = parse_input(test_input)
free_falling = False
resting = 0
while not free_falling:
    rock_map, free_falling = new_sand_unit_detect_freefall(rock_map, x_offset)
    if not free_falling:
        resting += 1
assert resting == 24

In [5]:
# Part 1

rock_map, x_offset = parse_input(Path("input.txt").read_text())
free_falling = False
resting = 0
while not free_falling:
    rock_map, free_falling = new_sand_unit_detect_freefall(rock_map, x_offset)
    if not free_falling:
        resting += 1

print(resting)

625


In [6]:
# Part 2 - test

rock_map, x_offset = parse_input(test_input, floor=True)
resting = 0
while rock_map[0][500-x_offset] == ".":
    rock_map = new_sand_unit(rock_map, x_offset)
    resting += 1

assert resting == 93

In [7]:
# Part 2

rock_map, x_offset = parse_input(Path("input.txt").read_text(), floor=True)
resting = 0
while rock_map[0][500-x_offset] == ".":
    rock_map = new_sand_unit(rock_map, x_offset)
    resting += 1

print(resting)

25193
